### Introduction:

This time you will create the data.



### Step 1. Import the necessary libraries

In [1]:
import numpy as np
from pyspark.ml.classification import LogisticRegression , DecisionTreeClassifier
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession , Row
from  pyspark.ml.linalg import Vectors
from pyspark.sql.types import *
from pyspark.sql.functions import col , when , lit , udf ,to_timestamp , monotonically_increasing_id 
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import pandas as pd

In [14]:
spark_sql = SparkSession.builder \
    .master('local[*]') \
    .appName('learn-spark') \
    .getOrCreate()
spark = spark_sql.sparkContext

#     .config("spark.driver.memory", "10g") \
#     .config("spark.executor.memory", "5g") \


In [29]:
data = spark_sql.read.option("header" , True).csv("Student-Pass-Fail-Data.csv")
data = data.withColumn('Self_Study_Daily' , data['Self_Study_Daily'].cast("float"))\
            .withColumn('Tution_Monthly' , data['Tution_Monthly'].cast("float"))\
            .withColumn('Pass_Or_Fail' , data['Pass_Or_Fail'].cast("float"))

data.show()
data.printSchema()

+----------------+--------------+------------+
|Self_Study_Daily|Tution_Monthly|Pass_Or_Fail|
+----------------+--------------+------------+
|             7.0|          27.0|         1.0|
|             2.0|          43.0|         0.0|
|             7.0|          26.0|         1.0|
|             8.0|          29.0|         1.0|
|             3.0|          42.0|         0.0|
|             5.0|          31.0|         0.0|
|             1.0|          44.0|         0.0|
|             7.0|          26.0|         1.0|
|             8.0|          27.0|         1.0|
|             4.0|          36.0|         0.0|
|             4.0|          37.0|         0.0|
|             8.0|          28.0|         1.0|
|             5.0|          20.0|         1.0|
|             3.0|          40.0|         0.0|
|             8.0|          29.0|         1.0|
|             3.0|          41.0|         0.0|
|             1.0|          49.0|         0.0|
|             3.0|          39.0|         0.0|
|            

In [30]:
assembler = VectorAssembler(inputCols = ["Self_Study_Daily" , "Tution_Monthly"] , outputCol="features")
data = assembler.transform(data)
data.show()

+----------------+--------------+------------+----------+
|Self_Study_Daily|Tution_Monthly|Pass_Or_Fail|  features|
+----------------+--------------+------------+----------+
|             7.0|          27.0|         1.0|[7.0,27.0]|
|             2.0|          43.0|         0.0|[2.0,43.0]|
|             7.0|          26.0|         1.0|[7.0,26.0]|
|             8.0|          29.0|         1.0|[8.0,29.0]|
|             3.0|          42.0|         0.0|[3.0,42.0]|
|             5.0|          31.0|         0.0|[5.0,31.0]|
|             1.0|          44.0|         0.0|[1.0,44.0]|
|             7.0|          26.0|         1.0|[7.0,26.0]|
|             8.0|          27.0|         1.0|[8.0,27.0]|
|             4.0|          36.0|         0.0|[4.0,36.0]|
|             4.0|          37.0|         0.0|[4.0,37.0]|
|             8.0|          28.0|         1.0|[8.0,28.0]|
|             5.0|          20.0|         1.0|[5.0,20.0]|
|             3.0|          40.0|         0.0|[3.0,40.0]|
|             

In [31]:
dataTrain = data.select("features" , col("Pass_Or_Fail").alias("label"))
dataTrain.show()

+----------+-----+
|  features|label|
+----------+-----+
|[7.0,27.0]|  1.0|
|[2.0,43.0]|  0.0|
|[7.0,26.0]|  1.0|
|[8.0,29.0]|  1.0|
|[3.0,42.0]|  0.0|
|[5.0,31.0]|  0.0|
|[1.0,44.0]|  0.0|
|[7.0,26.0]|  1.0|
|[8.0,27.0]|  1.0|
|[4.0,36.0]|  0.0|
|[4.0,37.0]|  0.0|
|[8.0,28.0]|  1.0|
|[5.0,20.0]|  1.0|
|[3.0,40.0]|  0.0|
|[8.0,29.0]|  1.0|
|[3.0,41.0]|  0.0|
|[1.0,49.0]|  0.0|
|[3.0,39.0]|  0.0|
|[6.0,24.0]|  1.0|
|[5.0,33.0]|  0.0|
+----------+-----+
only showing top 20 rows



In [66]:
model = LogisticRegression().fit(dataTrain)

In [77]:
model.predict(Vectors.dense([9.0,23.0]))

1.0

In [16]:
df_house = spark_sql.read.option("header" , True).csv("housing.csv")
df_house = df_house.drop("ocean_proximity" , "latitude" , "households" , "median_income" , "longitude")
df_house.show()

+------------------+-----------+--------------+----------+------------------+
|housing_median_age|total_rooms|total_bedrooms|population|median_house_value|
+------------------+-----------+--------------+----------+------------------+
|              41.0|      880.0|         129.0|     322.0|          452600.0|
|              21.0|     7099.0|        1106.0|    2401.0|          358500.0|
|              52.0|     1467.0|         190.0|     496.0|          352100.0|
|              52.0|     1274.0|         235.0|     558.0|          341300.0|
|              52.0|     1627.0|         280.0|     565.0|          342200.0|
|              52.0|      919.0|         213.0|     413.0|          269700.0|
|              52.0|     2535.0|         489.0|    1094.0|          299200.0|
|              52.0|     3104.0|         687.0|    1157.0|          241400.0|
|              42.0|     2555.0|         665.0|    1206.0|          226700.0|
|              52.0|     3549.0|         707.0|    1551.0|      

In [17]:
df_house = df_house.dropna()

In [18]:
df_house = df_house.withColumn("housing_median_age" ,df_house['housing_median_age'].cast('float'))\
        .withColumn("total_rooms" ,df_house['total_rooms'].cast('float'))\
        .withColumn("total_bedrooms" ,df_house['total_bedrooms'].cast('float'))\
        .withColumn("population" ,df_house['population'].cast('float'))\
        .withColumn("median_house_value" ,df_house['median_house_value'].cast('float'))
df_house.printSchema()

root
 |-- housing_median_age: float (nullable = true)
 |-- total_rooms: float (nullable = true)
 |-- total_bedrooms: float (nullable = true)
 |-- population: float (nullable = true)
 |-- median_house_value: float (nullable = true)



In [19]:
assembler_ln = VectorAssembler(inputCols = ["housing_median_age" , "total_rooms" , "total_bedrooms" , "population"] , outputCol="features")
df_house = assembler_ln.transform(df_house)
df_house.show()

+------------------+-----------+--------------+----------+------------------+--------------------+
|housing_median_age|total_rooms|total_bedrooms|population|median_house_value|            features|
+------------------+-----------+--------------+----------+------------------+--------------------+
|              41.0|      880.0|         129.0|     322.0|          452600.0|[41.0,880.0,129.0...|
|              21.0|     7099.0|        1106.0|    2401.0|          358500.0|[21.0,7099.0,1106...|
|              52.0|     1467.0|         190.0|     496.0|          352100.0|[52.0,1467.0,190....|
|              52.0|     1274.0|         235.0|     558.0|          341300.0|[52.0,1274.0,235....|
|              52.0|     1627.0|         280.0|     565.0|          342200.0|[52.0,1627.0,280....|
|              52.0|      919.0|         213.0|     413.0|          269700.0|[52.0,919.0,213.0...|
|              52.0|     2535.0|         489.0|    1094.0|          299200.0|[52.0,2535.0,489....|
|         

In [20]:
dataTrainHouse = df_house.select("features" , col("median_house_value").alias("label"))
dataTrainHouse.show()

+--------------------+--------+
|            features|   label|
+--------------------+--------+
|[41.0,880.0,129.0...|452600.0|
|[21.0,7099.0,1106...|358500.0|
|[52.0,1467.0,190....|352100.0|
|[52.0,1274.0,235....|341300.0|
|[52.0,1627.0,280....|342200.0|
|[52.0,919.0,213.0...|269700.0|
|[52.0,2535.0,489....|299200.0|
|[52.0,3104.0,687....|241400.0|
|[42.0,2555.0,665....|226700.0|
|[52.0,3549.0,707....|261100.0|
|[52.0,2202.0,434....|281500.0|
|[52.0,3503.0,752....|241800.0|
|[52.0,2491.0,474....|213500.0|
|[52.0,696.0,191.0...|191300.0|
|[52.0,2643.0,626....|159200.0|
|[50.0,1120.0,283....|140000.0|
|[52.0,1966.0,347....|152500.0|
|[52.0,1228.0,293....|155500.0|
|[50.0,2239.0,455....|158700.0|
|[52.0,1503.0,298....|162900.0|
+--------------------+--------+
only showing top 20 rows



In [21]:
modelPredictHouse = LinearRegression().fit(dataTrainHouse)

In [22]:
modelPredictHouse.predict(Vectors.dense([52.0 ,2491.0 , 474.0 , 1098.0 ]))

260994.4295859525

In [11]:
[i.features for i in dataTrainHouse.select('features').collect()]

[DenseVector([41.0, 880.0, 129.0, 322.0]),
 DenseVector([21.0, 7099.0, 1106.0, 2401.0]),
 DenseVector([52.0, 1467.0, 190.0, 496.0]),
 DenseVector([52.0, 1274.0, 235.0, 558.0]),
 DenseVector([52.0, 1627.0, 280.0, 565.0]),
 DenseVector([52.0, 919.0, 213.0, 413.0]),
 DenseVector([52.0, 2535.0, 489.0, 1094.0]),
 DenseVector([52.0, 3104.0, 687.0, 1157.0]),
 DenseVector([42.0, 2555.0, 665.0, 1206.0]),
 DenseVector([52.0, 3549.0, 707.0, 1551.0]),
 DenseVector([52.0, 2202.0, 434.0, 910.0]),
 DenseVector([52.0, 3503.0, 752.0, 1504.0]),
 DenseVector([52.0, 2491.0, 474.0, 1098.0]),
 DenseVector([52.0, 696.0, 191.0, 345.0]),
 DenseVector([52.0, 2643.0, 626.0, 1212.0]),
 DenseVector([50.0, 1120.0, 283.0, 697.0]),
 DenseVector([52.0, 1966.0, 347.0, 793.0]),
 DenseVector([52.0, 1228.0, 293.0, 648.0]),
 DenseVector([50.0, 2239.0, 455.0, 990.0]),
 DenseVector([52.0, 1503.0, 298.0, 690.0]),
 DenseVector([40.0, 751.0, 184.0, 409.0]),
 DenseVector([42.0, 1639.0, 367.0, 929.0]),
 DenseVector([52.0, 2436.0,

In [84]:
dfTitanic = spark_sql.read.option("header" , True).csv("titanic.csv")
dfTitanic.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [85]:
dfTitanic = dfTitanic.withColumn("Sex" , when(col("Sex") == "male", 1).otherwise(0))

In [86]:
dfTitanic.show()

+-----------+--------+------+--------------------+---+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+---+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  1|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|  0|  38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|  0|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|  0|  35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  1|  35|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  1|null|    0|    0|          330877| 8.4583| null|  

In [87]:
dfTitanic = dfTitanic.withColumn("Age" , col("Age").cast("float"))\
        .withColumn("Fare" , col("Fare").cast("float"))\
        .withColumn("Survived" , col("Survived").cast("float"))\
        .withColumn("Sex" , col("Survived").cast("float"))

In [88]:
dfTitanic = dfTitanic.dropna()
dfTitanic.show()


+-----------+--------+------+--------------------+---+----+-----+-----+-----------+--------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|Sex| Age|SibSp|Parch|     Ticket|    Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+---+----+-----+-----+-----------+--------+-----------+--------+
|          2|     1.0|     1|Cumings, Mrs. Joh...|1.0|38.0|    1|    0|   PC 17599| 71.2833|        C85|       C|
|          4|     1.0|     1|Futrelle, Mrs. Ja...|1.0|35.0|    1|    0|     113803|    53.1|       C123|       S|
|          7|     0.0|     1|McCarthy, Mr. Tim...|0.0|54.0|    0|    0|      17463| 51.8625|        E46|       S|
|         11|     1.0|     3|Sandstrom, Miss. ...|1.0| 4.0|    1|    1|    PP 9549|    16.7|         G6|       S|
|         12|     1.0|     1|Bonnell, Miss. El...|1.0|58.0|    0|    0|     113783|   26.55|       C103|       S|
|         22|     1.0|     2|Beesley, Mr. Lawr...|1.0|34.0|    0|    0|     248698|    1

In [89]:
assembler_dct = VectorAssembler(inputCols = ["Age" , "Sex" , "Fare"] , outputCol="features")
dfTitanic = assembler_dct.transform(dfTitanic)


In [90]:
dfTitanic_train = dfTitanic.select("features" , col("Survived").alias("label"))
dfTitanic_train.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[38.0,1.0,71.2833...|  1.0|
|[35.0,1.0,53.0999...|  1.0|
|[54.0,0.0,51.8624...|  0.0|
|[4.0,1.0,16.70000...|  1.0|
|[58.0,1.0,26.5499...|  1.0|
|     [34.0,1.0,13.0]|  1.0|
|     [28.0,1.0,35.5]|  1.0|
|    [19.0,0.0,263.0]|  0.0|
|[49.0,1.0,76.7292...|  1.0|
|[65.0,0.0,61.9791...|  0.0|
|[45.0,0.0,83.4749...|  0.0|
|     [29.0,1.0,10.5]|  1.0|
|[25.0,0.0,7.65000...|  0.0|
|    [23.0,1.0,263.0]|  1.0|
|[46.0,0.0,61.1749...|  0.0|
|[71.0,0.0,34.6542...|  0.0|
|[23.0,1.0,63.3582...|  1.0|
|[21.0,0.0,77.2874...|  0.0|
|     [47.0,0.0,52.0]|  0.0|
|[24.0,0.0,247.520...|  0.0|
+--------------------+-----+
only showing top 20 rows



In [91]:
model_dct = DecisionTreeClassifier().fit(dfTitanic_train)

In [96]:
model_dct.predict(Vectors.dense([23.0,1.0,263.0]))

1.0